# NER Powered Semantic Search 

### Setup Environment

In [ ]:
# init pinecone
import pandas as pd
import pinecone
from tqdm.autonotebook import tqdm # warning taken care of

# API_KEY = "YOUR API KEY"
# ENV = "YOUR ENVIRONMENT"

pinecone.init(api_key = API_KEY, environment = ENV)
index = pinecone.Index("medium-data")

In [ ]:
# clean up pinecone index
index.delete(delete_all=True)

In [ ]:
# delete index , dimensino no longer useful

In [2]:
# load libraries for NER 

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import torch

### NER Engine

In [3]:
# init NER engine

model_id = 'dslim/bert-base-NER'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForTokenClassification.from_pretrained(model_id)

# nlp pipeline

nlp = pipeline('ner',
              model=model,
              tokenizer=tokenizer,
              aggregation_strategy= 'max',
              device= 'cpu') 
# nlp("Bill Gates is the founder of Microsoft")

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Retriever

In [4]:
# load libraries for retriever

from sentence_transformers import SentenceTransformer


# https://huggingface.co/flax-sentence-embeddings/all_datasets_v3_mpnet-base
retriever = SentenceTransformer(
    "flax-sentence-embeddings/all_datasets_v3_mpnet-base")

In [5]:
retriever

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [6]:
# Create Pinecone Index
pinecone.create_index("medium-data", dimension= 768, metric="cosine")
index= pinecone.Index("medium-data")

### Data Prep

In [7]:
# Obtain Raw Data

# Method 1
# from datasets import load_dataset
# df = load_dataset(
#     "fabiochiu/medium-articles",
#     data_files="medium_articles.csv",
#     split="train"
# ).to_pandas()

# Method 2: Mannualy download the csv file
# https://huggingface.co/datasets/fabiochiu/medium-articles/tree/main
df = pd.read_csv('medium_articles.csv')

In [8]:
df = df.dropna().sample(1000, random_state=45) # might take 30mins to 1hr

df['text_extended'] = df['title'] + '.' + df['text'].str[:1000]

In [9]:
df.head()

,title,text,url,authors,timestamp,tags,text_extended
189059,How do you move a WordPress website to another...,Photo by Moritz Mentges on Unsplash\n\nMoving ...,https://medium.com/@dyderik/how-do-you-move-a-...,['Richard Detering'],2021-11-13 05:42:44.009000+00:00,"['Web Hosting Services', 'Web Hosting', 'Trans...",How do you move a WordPress website to another...
96618,A Long December,In my quest to find ways of engaging with the ...,https://medium.com/@keenekomeskleen/a-long-dec...,['Matt Keene'],2020-12-16 19:47:55.820000+00:00,"['Society', 'Politics', 'Poverty', 'Pandemic',...",A Long December.In my quest to find ways of en...
46027,I Have Decided to Stop Being the Michael Scott...,"Writing is, for me, a beloved pastime. I’ve do...",https://pisancantos43.medium.com/i-have-decide...,['Anthony Aycock'],2019-05-12 20:04:59.371000+00:00,"['Teaching', 'Television', 'College', 'Writing...",I Have Decided to Stop Being the Michael Scott...
145790,EU fully committed to sustainable development,European Commission Vice-President Jyrki Katai...,https://medium.com/ecajournal/eu-fully-committ...,['European Court Of Auditors'],2019-07-24 13:15:13.130000+00:00,['Sustainable Development'],EU fully committed to sustainable development....
132859,HD ▷..! เรื่องเต็ม 【M-Thai ดาบพิฆาตอสูร เดอะมู...,TAG::\n\nดาบพิฆาตอสูร เดอะมูฟวี่ ศึกรถไฟสู่นิร...,https://medium.com/@bangetanjay405/hd-%E0%B9%8...,[],2020-12-12 14:33:34.766000+00:00,"['Thailand', 'Japan', 'Taiwan', 'Hong Kong', '...",HD ▷..! เรื่องเต็ม 【M-Thai ดาบพิฆาตอสูร เดอะมู...


In [ ]:
# len(nlp(df_batch)) # list of lst

### NER Helper Function

In [10]:
# helper function for extracting entities of a batch of texts

def extract_entities(list_of_text):
    entities = []
    for doc in list_of_text: 
        entities.append([item['word'] for item in nlp(doc)])
        # list of entities for 1 doc
    return entities

In [ ]:
# embedding

# len(retriever.encode(df_batch))
# len(retriever.encode(df_batch[0])) # try for one doc
# embedding for batch
# emb = retriever.encode(df_batch).tolist() # array to python list

### Batch Upsert

In [11]:
# upsert data

from tqdm.auto import tqdm

batch_size = 64

for i in range(0, len(df), batch_size):
    i_end = min(i+batch_size, len(df))
    # print(i, i_end) # starting and ending index of each batch
    
    # get a batch of data
    df_batch = df.iloc[i: i_end].copy()
    
    # embedding
    emb = retriever.encode(df_batch['text_extended'].tolist()
                          ).tolist() # array to python list
    
    # ner extraction
    entities = extract_entities(df_batch['text_extended'].tolist())
    
    # [[]] --> [set1, set2, ], remove duplicate entities    
    df_batch['named_entity'] = [list(set(entity)) for entity in entities] # one list per document
    
    # create meta data
    df_batch = df_batch.drop('text', axis=1)
    
    meta_data = df_batch.to_dict(orient='records') # pd.df to dictionary
    
    # create ids
    
    ids = [f"{idx}" for idx in range(i, i_end)] #
    
    # upsert
    
    vectors_to_upsert = list(zip(ids, emb, meta_data))  # nd array to python list
    
    _ = index.upsert(vectors= vectors_to_upsert)  
    
    


In [ ]:
# index.describe_index_stats()

### Query data

In [12]:
query = "How to make a Wordpress website?"  # Natural Language

emb_qx = retriever.encode(query).tolist() # embedded query vector

ne = extract_entities([query])[0] # Named entity as a search filter

In [16]:
xc = index.query(emb_qx, top_k= 5, include_metadata=True,
           filter = {"named_entity": {"$in" : ne}})

In [17]:
for result in xc['matches']:
    print(result['score'], " ", result['metadata']['named_entity'])

0.294447154   ['Both Sides of the Table', 'Wordpress']
